#Trabajo Práctico Redes Neuronales
####Agustina Schajris Garati 4to C

Mi dataset consiste en varios casos de personas con COVID 19 y la gravedad de su condición. Mi tarea es predecir, dependiendo de los síntomas que tenga una persona, si la gravedad de su envermedad es leve, moderada, severa o nula. Es un problema de clasificación multiclase.

Mi dataset: [COVID-19 Symptoms Checker](https://www.kaggle.com/datasets/iamhungundji/covid19-symptoms-checker)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate
import networkx as nx
from random import random as rand
import itertools
import tensorflow as tf
import math

In [2]:
np.random.seed(1)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/Cleaned-Data.csv')
data

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,1,0,China
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,Other
316796,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,Other
316797,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,1,Other
316798,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,Other


In [5]:
#Función sigmoide, una de muchas funciones de activación. Sirve para que todo valor por debajo de 1/2 sea 0, y que todo valor sobre 1/2 sea 1; es una aproximación. Se utiliza para introducir no linealidad en la red.
def sigmoid(x):
    return 1 / ( 1 + np.exp(-x) )

#Función derivada de la sigmoide que se usa en el proceso de backpropagation para calcular los ajustes de los pesos.
def sigmoid_derivative(x):
    return x * (1 - x)

#Función de activación softmax para clasificación multiclase. Lo que hace es convertir un vector de valores de entrada en un vector de probabilidades(cada elemento del vector de salida representa la probabilidad de pertenencia a una clase específica).
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [6]:
columnas_input = list(range(0, 19)) + list(range(23, 26))
columnas_output = list(range(19, 23)) #multiclase

training_inputs = data.iloc[:, columnas_input]
training_outputs = data.iloc[:, columnas_output]

In [7]:
columnas_input

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 23, 24, 25]

In [8]:
columnas_output

[19, 20, 21, 22]

In [9]:
#Weights
n = training_inputs.shape[1] #número de entradas
m = training_outputs.shape[1] #número de salidas

#Fijo los weights de manera aleatoria cerca de cero. Después, esos números se van a ir ajustando en el entrenamiento.
synaptic_weights = 2 + np.random.random((n, m)) - 1

##Training

In [10]:
#500 epochs
for iteration in range(500):

    input_layer = training_inputs

    #FORWARD PROPAGATION
    #Multiplico los inputs por los weights y los paso por la función de activación softmax para obtener las salidas de la red.
    #Uso la función softmax ya que me permite convertir m salida en la probabilidad de pertenencia a cada clase. Es útil para tareas de clasfificación.
    outputs = np.dot(input_layer, synaptic_weights)
    softmax_outputs = softmax(outputs)

    #Calculo el error comparando las salidas reales y las salidas predichas para usarlo en backpropagation.
    error = training_outputs - softmax_outputs

    #BACKPROPAGATION
    #Uso el error para ajustar los weights usando la derivada de la función sigmoide. Uso la gradiente descendente para acercarme al punto con menor error más cercano, y así, ir ajustando los weights para hacer mejores predicciones.
    adjustments = error * sigmoid_derivative(softmax_outputs)
    synaptic_weights += np.dot(input_layer.T, adjustments)

In [11]:
print('Synaptic weights after training')
print(synaptic_weights)

print('Outputs after training: ')
print(outputs)

Synaptic weights after training
[[111.80990778 112.56232111 112.12252972 111.94136064]
 [111.80804797 112.13362577 112.71326779 112.28622946]
 [112.09463349 112.61230867 112.99632541 112.66157193]
 [111.86860155 112.93835306 112.55172128 112.62040152]
 [111.81308688 112.38455675 112.31306392 111.81337444]
 [  1.80260376   1.96898949   1.3155946    1.69318614]
 [112.38902274 112.79307439 112.24319367 111.57658735]
 [111.85069871 112.94292788 112.6228657  112.34616532]
 [112.65154682 112.59125219 113.24164832 112.25970139]
 [112.17571959 112.71601383 112.175393   112.50846792]
 [  1.99158479   1.74956834   1.28667934   1.79548329]
 [  2.15344102 104.09242287 108.65820614   2.46095011]
 [  2.78560196   2.51150638   1.58051794   5.94854934]
 [  2.49127158   3.1785044    2.63723748   1.84037958]
 [107.41445726   2.57779124   2.97317408 104.97708409]
 [  2.150945     5.12740152   3.45638251   2.90015528]
 [  1.62434168   3.07820465   3.33157091 109.9365047 ]
 [111.43927156   3.3265971  110.8

In [12]:
Fever = 1 #@param {type:"integer"}
Tiredness = 1 #@param {type:"integer"}
Dry_Cough = 0 #@param {type:"integer"}
Difficulty_in_Breathing = 1 #@param {type:"integer"}
Sore_Throat = 1 #@param {type:"integer"}
None_Sympton = 0 #@param {type:"integer"}
Pains = 1 #@param {type:"integer"}
Nasal_Congestion = 1 #@param {type:"integer"}
Runny_Nose = 0 #@param {type:"integer"}
Diarrhea = 0 #@param {type:"integer"}
None_Experiencing = 0 #@param {type:"integer"}
Age_0_9 = 1 #@param {type:"integer"}
Age_10_19 = 0 #@param {type:"integer"}
Age_20_24 = 0 #@param {type:"integer"}
Age_25_59 = 0 #@param {type:"integer"}
Age_60up = 1 #@param {type:"integer"}
Gender_Female = 0 #@param {type:"integer"}
Gender_Male = 1 #@param {type:"integer"}
Gender_Transgender = 0 #@param {type:"integer"}
Contact_Dont_Know = 0 #@param {type:"integer"}
Contact_No = 0 #@param {type:"integer"}
Contact_Yes = 1 #@param {type:"integer"}


input_layer = np.array([Fever, Tiredness, Dry_Cough, Difficulty_in_Breathing,
       Sore_Throat, None_Sympton, Pains, Nasal_Congestion,
       Runny_Nose, Diarrhea, None_Experiencing, Age_0_9, Age_10_19,
       Age_20_24, Age_25_59, Age_60up, Gender_Female, Gender_Male,
       Gender_Transgender, Contact_Dont_Know, Contact_No, Contact_Yes])

outputs = np.dot(input_layer, synaptic_weights)
softmax_outputs = softmax(outputs) #uso la función softmax para porder sacar las probabilidades

probability_mild = math.trunc(softmax_outputs[0] * 100) / 100
probability_moderate = math.trunc(softmax_outputs[1] * 100) / 100
probability_none = math.trunc(softmax_outputs[2] * 100) / 100
probability_severe = math.trunc(softmax_outputs[3] * 100) / 100

print("Probability mild: " + str(probability_mild))
print("Probability moderate: " + str(probability_moderate))
print("Probability severe: " + str(probability_severe))
print("Probability none: " + str(probability_none))

Probability mild: 0.0
Probability moderate: 0.17
Probability severe: 0.0
Probability none: 0.82
